In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
from heapq import nsmallest
from scipy.signal import butter, lfilter, freqz
from scipy.fftpack import fft, ifft

In [2]:
combine_parcial =  pd.read_csv('zetaAur-eclipse_B_2019_10_25_edit.csv', delimiter = '\t', header = 0)
combine_parcial.columns = ['L','I', 'C']
combine_parcial['L'] = combine_parcial['L']*10
combine_parcial['I'] = (combine_parcial['I']*1.8 + 0.5)

In [3]:
def get_fs(Data):
    difer = 0
    for i in range(1,len(combine_parcial['L'])): 
        difer += combine_parcial['L'][i] - combine_parcial['L'][i-1]
    return difer/(len(combine_parcial['L'])-1)
               
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_lowpass(highcut, fs, order=5):
    nyq = 0.5 * fs
    high = highcut / nyq
    b, a = butter(order, high, btype='low')
    return b, a


def butter_lowpass_filter(data, highcut, fs, order=5):
    b, a = butter_lowpass( highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def FFT(Data,fs):
    N = len(Data)
    T = 1/fs
    yf = fft(Data)
    xf = np.linspace(0.0, 1.0/(2.0*T), N//2)
    return [xf, yf, N]

In [5]:
dx = np.diff(combine_parcial['I'])/np.diff(combine_parcial['L'])
dy = dx.tolist()
dy.append(0)

In [6]:
fs = get_fs(dy)
newdy = butter_lowpass_filter(dy,0.01,fs,5)
highpass_Inte = butter_bandpass_filter(combine_parcial['I'],0.00008,0.03,fs,5)

In [ ]:
def fast_peaks():